In [1]:
from modules.ingestion.chunking import llmsherpa
from modules.ingestion.embedding import huggingface
from IPython.core.display import HTML

name_file_pdf = 'Car_Diagnostics_Hackathon.pdf'

sherpa = llmsherpa.ChunkText() 
hf_embedder = huggingface.Embedder(model_url='sentence-transformers/all-MiniLM-L6-v2',
                                   batch_size=8)

c:\Users\ASUS\anaconda3\envs\multimodel_rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
docs = sherpa.chunking(name_file_pdf, get_section_header=True, get_small_chunks=False)
hf_embedder.embedding_documents(docs)

True

In [15]:
import weaviate
import json

WEAVIATE_URL = "https://multimodel-rag-7opyhshr.weaviate.network"
WEAVIATE_API_KEY = "EBWshsUYap5giogjUaPyPbElUYW4pKf3o86O"

client = weaviate.connect_to_wcs(
    cluster_url=WEAVIATE_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_API_KEY),
    skip_init_checks=True
)


In [18]:
class_name = 'test_rag'

if client.schema.exists(class_name):
   client.schema.delete_class(class_name)

class_obj = {
    "class": class_name,
    "vectorizer": "none"
    "pq": {
        "enabled": True,  # Enable PQ
        "trainingLimit": 50000  # Set the threshold to begin training
    }
}

client.schema.create_class(class_obj)

SyntaxError: invalid syntax (1998187490.py, line 9)

In [10]:
client.batch.configure(batch_size=50)  # Configure batch

with client.batch as batch:  # Initialize a batch process

    for i, doc in enumerate(docs):  # Batch import data

        properties = {
            "tag": doc["tag"],
            "level": doc["page_idx"],
            "file_name": doc["filename"],
            "text": doc["text"],
            "hmtl_text": doc["html_text"]
        }
        batch.add_data_object(
            data_object=properties,
            class_name="test_rag",
            vector=doc['vector']
        )